<a href="https://colab.research.google.com/github/Yayunyun/IDS705_final_project/blob/main/training_testing_biased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title import module and load data from google drive
# connect to google drive
from google.colab import drive
drive.mount('/content/drive/', force_remount = False)
# cd to my final project folder
%cd /content/drive/Othercomputers/My_MacBook_Pro/final_project/data
# move to working folder in colab
%cd /content
%mkdir final_project
%cd final_project

# unzip folder from google drive to colab
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images.zip
#!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images2.zip
#!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images3.zip
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images4.zip
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images5.zip
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images6.zip
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images7.zip
#!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images8.zip
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images9.zip
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images10.zip
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images11.zip
!unzip /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/images12.zip

# copy files from g-drive to colab
%cp /content/drive/Othercomputers/My_MacBook_Pro/final_project/data/Data_Entry_2017.csv .
# # import pacakges
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

%matplotlib inline

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import os
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

import os
import pandas as pd
from torchvision.io import read_image
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

%matplotlib inline
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

### OpenMP issues
os.environ['KMP_DUPLICATE_LIB_OK']='True'

%cd /content

cuda:0
PyTorch Version:  2.0.0+cu118
Torchvision Version:  0.15.1+cu118
/content


# Model development

read data

In [ ]:
df_imbalanced_20_80 = pd.read_pickle('/content/drive/Othercomputers/My_MacBook_Pro/final_project/df30_70.pkl')
test_df_ = pd.read_pickle('/content/drive/Othercomputers/My_MacBook_Pro/final_project/df_test.pkl')

In [ ]:
df_imbalanced_20_80.head()

,Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],FilePath,Sick,gender
85882,00021199_000.png,Infiltration,0,21199,44,M,PA,3056,2544,0.139,0.139,./final_project/images10/00021199_000.png,1,1
28230,00007354_002.png,No Finding,2,7354,42,M,AP,2500,2048,0.168,0.168,./final_project/images4/00007354_002.png,0,1
34533,00009093_006.png,No Finding,6,9093,26,M,PA,2742,2991,0.143,0.143,./final_project/images4/00009093_006.png,0,1
21350,00005694_000.png,No Finding,0,5694,45,M,AP,2500,2048,0.171,0.171,./final_project/images3/00005694_000.png,0,1
78538,00019271_021.png,Atelectasis|Infiltration,21,19271,54,M,AP,3056,2544,0.139,0.139,./final_project/images8/00019271_021.png,1,1


In [ ]:
# initializae hyperparameter
model_name = "alexnet"
num_classes = 2
batch_size = 16
num_epochs = 4
# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

## 2. Create Data Loader for the training and the validation set
Returns the image, label and the gender of the person who has taken the X-Ray. 

In [ ]:
#@title function to create gender-imablanced data
def create_by_ratio(ratio, train_df):

  """function to create gender-imablanced data
  ratio: female:male ratio
  train_df: dataframe to use"""

  # make an imbalanced dataset by undersampling female class
  train_male = train_df[train_df['Patient Gender'] == 'M']
  train_female = train_df[train_df['Patient Gender'] == 'F']
  # heart ==1
  male_sample_num = train_male[train_male['Sick'] == 1].shape[0]
  female_sample_num = np.round(male_sample_num*ratio).astype(int)
  female_df1 = train_female[train_female['Sick'] == 1].sample(female_sample_num,  random_state = 1)


  df_imbalanced = pd.concat([train_male, female_df1])
  df_imbalanced = pd.concat([df_imbalanced, train_female[train_female['Sick'] == 0]])

  return df_imbalanced


In [ ]:
#@title Create dataloader
class CustomImageDataset(Dataset):
    def __init__(self, finding, sensitive_attribute, path, transform=None, target_transform=None):
        self.img_labels = finding
        self.transform = transform
        self.target_transform = target_transform
        self.sensitive_attribute = sensitive_attribute
        self.path = path
    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = self.path.iloc[idx]
        image = Image.open(img_path).convert('RGB') ## change to RGB cuz resnet, densenet only take RGB
        label = self.img_labels.iloc[idx]
        sensitive_attr = self.sensitive_attribute.iloc[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        sample = {'image': image, 'label': label, 'sensitive_attr':sensitive_attr}
        return sample 
    
transform = transforms.Compose([
     transforms.Resize((224, 224)),
    transforms.ToTensor()
])

df_train, df_val = train_test_split(df_imbalanced_20_80, test_size=0.2, random_state=1)

train_dataset = CustomImageDataset(df_train['Sick'], df_train['gender'], 
                                df_train['FilePath'], transform=transform)
val_dataset = CustomImageDataset(df_val['Sick'], df_val['gender'], 
                                df_val['FilePath'], transform=transform)
df_balanced_test = create_by_ratio(0.5, test_df_)
test_dataset = CustomImageDataset(df_balanced_test['Sick'], df_balanced_test['gender'], 
                                df_balanced_test['FilePath'], transform=transform)


train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)

val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle = True)


combined_data_loader = {"train":train_data_loader, "val":val_data_loader}

# Define the arcitecture of the model and the loss function

In [1]:
#@title Define fairloss function
# SPDX-License-Identifier: GPL-3.0-only
# SPDX-FileCopyrightText: 2020 Vincent Lequertier <vi.le@autistici.org>

import torch
from typing import Callable, Union


class FairLoss2(torch.nn.Module):
    def __init__(
        self,
        loss_fun: torch.nn.Module,
        unique_attr: torch.Tensor,
        fairness_score: Union[
            str, Callable[[torch.Tensor, torch.Tensor], torch.Tensor]],
        strength = 0
    ) -> None:
        """
        Add a fairness measure to the regular loss

        fairness_score is applied to input and target for each value of
        unique_attr. Then the results are sumed up, divided by the minimum and
        added to the regular loss function.


        .. math::
           loss + \\lambda{{\\sum_{i=0}^{k} w_i f_i(input, target)} \\over \\min\\limits_{ \\forall i\\in [0,k[} f_i(input, target)}

        where:

        - :math:`k` is the number of values of ``protected_attr``
        - :math:`f` is the ``fairness_score`` function


        Args:
            loss_fun (torch.nn.Module): A loss function
            unique_attr (torch.Tensor): Possible values of a sensitive attribute
            fairness_score (Union[str, Callable[[torch.Tensor, torch.Tensor], torch.Tensor]]): A function that takes input and target as arguments and return a score. Or one of 'accuracy', 'fpr', 'tpr', 'tnr', 'fnr', 'ppv', 'npv', 'accuracy'

        Examples:
            >>> model = Model()
            >>> data = torch.randint(0, 5, (100, 5), dtype=torch.float, requires_grad=True)
            >>> target = torch.randint(0, 5, (100, 1), dtype=torch.float)
            >>> input = model(data)
            >>> # The sensitive attribute is the second column
            >>> dim = 1
            >>> criterion = FairLoss(torch.nn.MSELoss(), data[:, dim].detach().unique(), 'accuracy')
            >>> loss = criterion(data[:, dim], y_pred, y_true)
        """

        super().__init__()
        self.loss = loss_fun
        self.unique_attr = unique_attr
        self.fairness_score = (
            self.get_fairness_score(fairness_score)
            if isinstance(fairness_score, str)
            else fairness_score
        )
        self.strength = strength

    def forward(
        self, protected_attr: torch.Tensor, input: torch.Tensor, target: torch.Tensor
    ):
        """
        Compute the fair loss

        Shape:
            - protected_attr: :math:`(N,)`
            - input: :math:`(N, 1)`
            - target: :math:`(N, 1)`

        Returns:
            torch.Tensor: The fair loss value
        """
        scores = []

        for val in self.unique_attr:
          fair_input = input[torch.where(protected_attr == val)].reshape(-1,)
          fair_target = target[torch.where(protected_attr == val)].reshape(-1,)
          scores.append(self.fairness_score(fair_input, fair_target))
        scores = torch.FloatTensor(scores)
        

        # Sum up and divide by the minimum. Then add to the regular loss
        target = torch.reshape(target, (-1,1)).float()
        loss2 = scores.sum() / (scores.min() + 1e-7)
        if torch.isnan(loss2) == True:
          loss2 = torch.FloatTensor([0]).cuda()
        return_loss = torch.add(self.loss(input, target), self.strength * loss2) 
        return self.loss(input, target)


    def get_fairness_score(
        self,
        fairness_score: str,
    ) -> Callable[[torch.Tensor, torch.Tensor], torch.Tensor]:
        """
        Return one of the fairness scores that are built-in

        Args:
            fairness_score (str): The fairness score

        Returns:
            Callable[[torch.Tensor, torch.Tensor], torch.Tensor]: The fairness score function
        """
        if hasattr(self, fairness_score):
            return getattr(self, fairness_score)
        else:
            raise ValueError(
                'The fairness score "{}" is unavailable'.format(fairness_score)
            )


    @staticmethod
    def fpr(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        False Positive Rate

        .. math::
           {FPR} = {FP \\over FP + TN}

        where:

        - :math:`FP` is the number of False Positive
        - :math:`TN` is the number of True Negative

        Args:
            input (torch.Tensor): Predicted values
            target (torch.Tensor): Ground truth

        Shape:
            - input: :math:`(N, 1)`
            - target: :math:`(N, 1)`

        Returns:
            torch.Tensor: False Positive Rate

        Examples:
            >>> input = np.random.randint(2, size=(10, 1)).astype("float")
            >>> input = torch.tensor(input)
            >>> target = np.random.randint(2, size=(10, 1)).astype("float")
            >>> target = torch.tensor(target)
            >>> fpr(input, target)
        """
        fp = sum((input == True) & (target == False))
        tn = sum((input == False) & (target == False))
        return torch.true_divide(fp, fp + tn)


    @staticmethod
    def tpr(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        True Positive Rate

        .. math::
           {TPR} = {TP \\over TP + FN}

        where:

        - :math:`TP` is the number of True Positive
        - :math:`FN` is the number of False Negative

        Args:
            input (torch.Tensor): Predicted values
            target (torch.Tensor): Ground truth

        Shape:
            - input: :math:`(N, 1)`
            - target: :math:`(N, 1)`

        Returns:
            torch.Tensor: True Positive Rate

        Examples:
            >>> input = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> target = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> tpr(input, target)
        """
        fn = sum((input == False) & (target == True))
        tp = sum((input == True) & (target == True))
        return torch.true_divide(tp, tp + fn)


    @staticmethod
    def tnr(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        True Negative Rate

        .. math::
           {TNR} = {TN \\over TN + FP}

        where:

        - :math:`TN` is the number of True Negative
        - :math:`FP` is the number of False Positive

        Args:
            input (torch.Tensor): Predicted values
            target (torch.Tensor): Ground truth

        Shape:
            - input: :math:`(N, 1)`
            - target: :math:`(N, 1)`

        Returns:
            torch.Tensor: True Negative Rate

        Examples:
            >>> input = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> target = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> tnr(input, target)
        """
        fp = sum((input == True) & (target == False))
        tn = sum((input == False) & (target == False))
        return torch.true_divide(tn, tn + fp)


    @staticmethod
    def fnr(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        False Negative Rate

        .. math::
           {FNR} = {FN \\over FN + TP}

        where:

        - :math:`FN` is the number of False Negative
        - :math:`TP` is the number of True Positive

        Args:
            input (torch.Tensor): Predicted values
            target (torch.Tensor): Ground truth

        Shape:
            - input: :math:`(N, 1)`
            - target: :math:`(N, 1)`

        Returns:
            torch.Tensor: False Negative Rate

        Examples:
            >>> input = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> target = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> fnr(input, target)
        """
        input = torch.round(input)
        target = torch.round(target)
        fn = sum((input == 0) & (target == 1))
        tp = sum((input == 1) & (target == 1))
        return torch.true_divide(fn, fn + tp)


    @staticmethod
    def ppv(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Positive Predicted Value

        .. math::
           {PPV} = {TP \\over TP + FP}

        where:

        - :math:`TP` is the number of True Positive
        - :math:`FP` is the number of False Positive

        Args:
            input (torch.Tensor): Predicted values
            target (torch.Tensor): Ground truth

        Shape:
            - input: :math:`(N, 1)`
            - target: :math:`(N, 1)`

        Returns:
            torch.Tensor: Positive Predicted Value

        Examples:
            >>> input = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> target = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> ppv(input, target)
        """
        tp = sum((input == True) & (target == True))
        fp = sum((input == True) & (target == False))
        return torch.true_divide(tp, tp + fp)


    @staticmethod
    def npv(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Negative Predicted Value

        .. math::
           {NPV} = {TN \\over TN + FN}

        where:

        - :math:`TN` is the number of True Negative
        - :math:`FN` is the number of False Negative

        Args:
            input (torch.Tensor): Predicted values
            target (torch.Tensor): Ground truth

        Shape:
            - input: :math:`(N, 1)`
            - target: :math:`(N, 1)`

        Returns:
            torch.Tensor: Negative Predicted Value

        Examples:
            >>> input = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> target = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> npv(input, target)
        """
        tn = sum((input == False) & (target == False))
        fn = sum((input == False) & (target == True))
        return torch.true_divide(tn, tn + fn)


    @staticmethod
    def accuracy(input: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Accuracy

        Args:
            input (torch.Tensor): Predicted values
            target (torch.Tensor): Ground truth

        Shape:
            - input: :math:`(N, 1)`
            - target: :math:`(N, 1)`

        Returns:
            torch.Tensor: Accuracy

        Examples:
            >>> input = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> target = torch.randint(0, 2, (10, 1), dtype=torch.float)
            >>> accuracy(input, target)
        """
        input = torch.round(input)
        target = torch.round(target)
        return torch.true_divide((input == target).sum(), input.shape[0])

In [ ]:
#@title Define training code function
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    #params for early stopping
    prev_val_loss = 0
    stop_patience = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                current_dataloader = dataloaders[phase]
                loop = tqdm(current_dataloader)
            else:
                model.eval()   # Set model to evaluate mode
                current_dataloader = dataloaders[phase]
                loop = tqdm(current_dataloader)
            running_loss = 0.0
            running_corrects = 0
        
            # Iterate over data.
            for idx, data in enumerate(loop):

                inputs = data['image'].to(device)
                labels = data['label'].to(device)
                
                sensitive_attr = torch.clone(torch.tensor(data['sensitive_attr']))
                
                inputs = inputs.to(device)
                

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                        print('inception', loss)
                    else:
                        outputs = model(inputs)
                        #loss = criterion(outputs, labels)
                        loss = criterion(sensitive_attr, outputs, labels) ############# Changed here
                        # print(f'batch loss: {loss:.2f}')
                    
                    # _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                # running_corrects += torch.sum(preds == labels.data)
                running_corrects += torch.sum(torch.round(outputs).reshape(-1,1) == labels.reshape(-1,1)).item()
                loop.set_description(f"Epoch[{epoch+1}/{num_epochs}]  - {phase}")
                # loop.set_postfix(loss= 12, acc=12)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

            # ## early stopping
            # if phase =='val' and epoch_loss > prev_val_loss and prev_val_loss !=0:
            #   prev_val_loss = epoch_loss
            #   stop_patience += 1
            #   if stop_patience == 2:
            #     break
            # else: 
            #   prev_val_loss = epoch_loss

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            

In [ ]:
#@title Function for model initialization
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if num_classes == 2:
      num_classes = 1 # set loss function to BCEWithLogitsLoss()

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,1)
        model_ft.features[0] = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft1, input_size = initialize_model('alexnet', num_classes, feature_extract, use_pretrained=True)
model_ft1.classifier = nn.Sequential(model_ft1.classifier, nn.Sigmoid())


In [ ]:
# Send the model to GPU
model_ft1 = model_ft1.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft1.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft1.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft1.named_parameters():
        if param.requires_grad == True:
          pass
          print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)


In [ ]:
# Setup the loss fxn
criterion = FairLoss2(nn.BCELoss(), [1, 0], 'fnr') ########change here
model_ft1, hist = train_model(model_ft1, combined_data_loader, criterion, optimizer_ft, num_epochs= 15, is_inception=(model_name=="inception"))

Epoch 1/15
----------


  0%|          | 0/793 [00:00<?, ?it/s]<ipython-input-120-45654fcfe056>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sensitive_attr = torch.clone(torch.tensor(data['sensitive_attr']))
Epoch[1/15]  - train: 100%|██████████| 793/793 [04:18<00:00,  3.07it/s]


train Loss: 0.6621 Acc: 0.6080


Epoch[1/15]  - val: 100%|██████████| 199/199 [01:03<00:00,  3.14it/s]


val Loss: 0.6778 Acc: 0.6233

Epoch 2/15
----------


...

val Loss: 0.5848 Acc: 0.6970

Epoch 15/15
----------


Epoch[15/15]  - train: 100%|██████████| 793/793 [04:17<00:00,  3.08it/s]


train Loss: 0.5323 Acc: 0.7333


Epoch[15/15]  - val: 100%|██████████| 199/199 [01:03<00:00,  3.13it/s]

val Loss: 0.5967 Acc: 0.6948

Training complete in 80m 22s
Best val Acc: 0.698928


In [ ]:
# # save model
torch.save(model_ft1.state_dict(), \
           '/content/drive/Othercomputers/My_MacBook_Pro/final_project/torch_model_8020.pth')

# # load model
# new_model, _ = initialize_model('alexnet',1, False, use_pretrained=False)
# new_model.classifier = nn.Sequential(new_model.classifier, nn.Sigmoid())
# new_model.load_state_dict(torch.load("/content/drive/Othercomputers/My_MacBook_Pro/final_project/torch_model_8020.pth"))


# Predictions on different test sets

In [ ]:
#@title function to get different metrics on the test set
def get_metricss(logits, ratio, df_balanced_test):
  """function to get metrics on the test set
  logits: output from the model
  ratio: label for the gender-imablance test set
  df_balanced_test: test data"""

  predict_classes = np.round(logits) # use 0.5 as cut-off value
  predict_classes = predict_classes
  df_balanced_test['predict_classes'] = predict_classes

  # evaluate overall 
  precision_ = precision_score(np.array(df_balanced_test['Sick'], dtype='int64').reshape(-1,), predict_classes)
  accuracy_ = accuracy_score(np.array(df_balanced_test['Sick'], dtype='int64').reshape(-1,), predict_classes)
  recall_ = recall_score(np.array(df_balanced_test['Sick'], dtype='int64').reshape(-1,), predict_classes)
  f1_ = f1_score(np.array(df_balanced_test['Sick'], dtype='int64').reshape(-1,), predict_classes)
  fpr = len(df_balanced_test[(df_balanced_test['predict_classes'] == 1)& (df_balanced_test['Sick'] == 0)])/len(df_balanced_test[df_balanced_test['Sick'] == 0])
  fnr = len(df_balanced_test[(df_balanced_test['predict_classes'] == 0)& (df_balanced_test['Sick'] == 1)])/len(df_balanced_test[df_balanced_test['Sick'] == 1])



  # evaluate preformance for male and female
  df_balanced_test['predict_classes'] = predict_classes
  male_test = df_balanced_test[df_balanced_test['Patient Gender'] == 'M']
  female_test = df_balanced_test[df_balanced_test['Patient Gender'] == 'F']

  male_precision = precision_score(np.array(male_test['Sick'], dtype='int64').reshape(-1,), male_test['predict_classes'])
  female_precision = precision_score(np.array(female_test['Sick'], dtype='int64').reshape(-1,), female_test['predict_classes'])

  male_accuracy = accuracy_score(np.array(male_test['Sick'], dtype='int64').reshape(-1,), male_test['predict_classes'])
  female_accuracy = accuracy_score(np.array(female_test['Sick'], dtype='int64').reshape(-1,), female_test['predict_classes'])

  male_recall = recall_score(np.array(male_test['Sick'], dtype='int64').reshape(-1,), male_test['predict_classes'])
  female_recall = recall_score(np.array(female_test['Sick'], dtype='int64').reshape(-1,), female_test['predict_classes'])

  male_f1 = f1_score(np.array(male_test['Sick'], dtype='int64').reshape(-1,), male_test['predict_classes'])
  female_f1 = f1_score(np.array(female_test['Sick'], dtype='int64').reshape(-1,), female_test['predict_classes'])

  male_fpr = len(male_test[(male_test['predict_classes'] == 1)& (male_test['Sick'] == 0)])/len(male_test[male_test['Sick'] == 0])
  male_fnr = len(male_test[(male_test['predict_classes'] == 0)& (male_test['Sick'] == 1)])/len(male_test[male_test['Sick'] ==1])


  female_fpr = len(female_test[(female_test['predict_classes'] == 1)& (female_test['Sick'] == 0)])/len(female_test[female_test['Sick'] == 0])
  female_fnr = len(female_test[(female_test['predict_classes'] == 0)& (female_test['Sick'] == 1)])/len(female_test[female_test['Sick'] ==1])

  fairness_metric = (((len(female_test)/len(df_balanced_test))*female_fnr) + ((len(male_test)/len(df_balanced_test))* male_fnr))/min(female_fnr, male_fnr)
  results = [ratio, precision_, male_precision, female_precision,\
                           accuracy_, male_accuracy, female_accuracy, \
                           recall_, male_recall, female_recall, \
                           f1_, male_f1, female_f1, \
                          fpr, male_fpr, female_fpr, \
                          fnr, male_fnr, female_fnr, fairness_metric]
  return results                          

In [ ]:
# predict on 50 50 test set
test_data_loader = DataLoader(test_dataset, batch_size= 64, shuffle = False, num_workers = 10)
x = np.array([])
for data in test_data_loader:
  inputs =  data['image'].cuda()
  outputs = model_ft1(inputs).detach().cpu().numpy().reshape(-1,)
  x = np.concatenate((x, outputs), axis = 0)
results_70_1 = get_metricss(x, 'M/F normal model train:70/30 test: 50/50', df_balanced_test)


# predict on 70 30 test set
df_balanced_test1 = create_by_ratio(30/70, test_df_)
test_dataset_3070 = CustomImageDataset(df_balanced_test1['Sick'], df_balanced_test1['gender'], 
                                df_balanced_test1['FilePath'], transform=transform)
test_data_loader1 = DataLoader(test_dataset_3070, batch_size= 64, shuffle = False, num_workers = 10)
x2 = np.array([])
for data in test_data_loader1:
  inputs =  data['image'].cuda()
  outputs = model_ft1(inputs).detach().cpu().numpy().reshape(-1,)
  x2 = np.concatenate((x2, outputs), axis = 0)

results_70_2 = get_metricss(x2, 'M/F normal model train:70/30 test: 70/30', df_balanced_test1)

# predict on 90 10 test set
df_balanced_test2 = create_by_ratio(10/90, test_df_)
test_dataset_1090 = CustomImageDataset(df_balanced_test2['Sick'], df_balanced_test2['gender'], 
                                df_balanced_test2['FilePath'], transform=transform)
test_data_loader2 = DataLoader(test_dataset_1090, batch_size= 64, shuffle = False, num_workers = 10)
x3 = np.array([])
for data in test_data_loader2:
  inputs =  data['image'].cuda()
  outputs = model_ft1(inputs).detach().cpu().numpy().reshape(-1,)
  x3 = np.concatenate((x3, outputs), axis = 0)

results_70_3 = get_metricss(x3, 'M/F normal model train:70/30 test: 90/10', df_balanced_test2)

In [ ]:
results_df = pd.DataFrame(columns = ['male/female', 'precision', 'male_precision', 'female_precision',\
                                    'accuracy', 'male_accuracy', 'female_accuracy', \
                                    'recall', 'male_recall', 'female_recall',\
                                    'f1', 'male_f1', 'female_f1', 'fpr', 'male_fpr', 'female_fpr', \
                                     'fnr', 'male_fnr', 'female_fnr', 'fairness_metric'])
results_df.loc[1] = results_70_1
results_df.loc[2] = results_70_2
results_df.loc[3] = results_70_3

In [ ]:
results_df

,male/female,precision,male_precision,female_precision,accuracy,male_accuracy,female_accuracy,recall,male_recall,female_recall,f1,male_f1,female_f1,fpr,male_fpr,female_fpr,fnr,male_fnr,female_fnr,fairness_metric
1,M/F normal model train:70/30 test: 50/50,0.659162,0.685883,0.594255,0.664980,0.670189,0.656857,0.609685,0.673929,0.481105,0.633459,0.679853,0.531727,0.285026,0.333857,0.223541,0.390315,0.326071,0.518895,1.231022
2,M/F normal model train:70/30 test: 70/30,0.650564,0.685883,0.556863,0.669254,0.670189,0.667708,0.616167,0.673929,0.481356,0.632898,0.679853,0.516364,0.285026,0.333857,0.223541,0.383833,0.326071,0.518644,1.222425
3,M/F normal model train:70/30 test: 90/10,0.605455,0.685883,0.239057,0.690089,0.670189,0.735395,0.652941,0.673929,0.464052,0.628302,0.679853,0.315556,0.285026,0.333857,0.223541,0.347059,0.326071,0.535948,1.196437


In [ ]:
results_df.to_csv('/content/drive/Othercomputers/My_MacBook_Pro/final_project/result_8020.csv')